In [1]:
import collections
import math
import heapq
from typing import *

# 5259. Calculate Amount Paid in Taxes

In [2]:
class Solution:
    def calculateTax(self, brackets: List[List[int]], income: int) -> float:
        if income >= brackets[0][0]:
            ret = brackets[0][0] * brackets[0][1]  
        else:
            return income * brackets[0][1] /100
        for i in range(1, len(brackets)):
            if income >= brackets[i][0]:
                ret += (brackets[i][0]-brackets[i-1][0]) * brackets[i][1]
            else:
                ret += (income-brackets[i-1][0]) * brackets[i][1]
                break
        return ret/100

# 5270. Minimum Path Cost in a Grid

In [60]:
class Solution:
    def minPathCost(self, grid: List[List[int]], moveCost: List[List[int]]) -> int:
        pre = grid[0]
        for i in range(len(grid)-1):
            nxt = [float("inf")] * len(grid[0])
            for s in range(len(grid[0])):
                for j in range(len(grid[0])):
                    nxt[j] = min(nxt[j], pre[s] + moveCost[grid[i][s]][j] + grid[i+1][j])
            pre = nxt
        return min(pre)

In [61]:
grid = [[5,1,2],[4,0,3]]
moveCost = [[12,10,15],[20,23,8],[21,7,1],[8,1,13],[9,10,25],[5,3,2]]

In [62]:
a = Solution().minPathCost(grid, moveCost)
a

6

# 5289. Fair Distribution of Cookies

In [20]:
import functools
class Solution:
    def distributeCookies(self, cookies: List[int], k: int) -> int:
        def xunzhao(i, people):
            if i == len(cookies):
                return min(people)
            ret = float("inf")
            people = list(people)
            for j in range(k):
                people[j] += cookies[i]
                ret = min(ret, xunzhao(i+1, tuple(people)))
                people[j] -= cookies[i]
            return ret
        
        return xunzhao(0, tuple([0]*k))

In [63]:
import functools
class Solution:
    def distributeCookies(self, cookies: List[int], k: int) -> int:
        avg = math.ceil(sum(cookies)/k)
        def xunzhao(i, people):
            if i == len(cookies):
                return max(people)
            ret = float("inf")
            for j in range(k):
                if people[j] >= avg:
                    continue
                people[j] += cookies[i]
                ret = min(ret, xunzhao(i+1, people))
                people[j] -= cookies[i]
            return ret
        
        return xunzhao(0, [0]*k)

In [67]:
cookies = [8,15,10,20,8]
k = 2

子集状态压缩

In [89]:
import copy
class Solution:
    def distributeCookies(self, cookies: List[int], k: int) -> int:
        lens = 1 << len(cookies)
        sums = [0] * lens
        for i, cookie in enumerate(cookies):
            bits = 1<<i
            for i in range(bits):
                sums[i|bits] = sums[i] + cookie
        f = copy.deepcopy(sums)
        for i in range(1, k):
            for j in range(lens-1, 0, -1):
                x = j
                while x:
                    f[j] = min(f[j], max(f[x^j], sums[x]))
                    x = (x-1)&j
        return f[-1]

In [90]:
a = Solution().distributeCookies(cookies, k)
a

31

# 6094. Naming a Company

In [92]:
#wrong answer
class Solution:
    def distinctNames(self, ideas: List[str]) -> int:
        re = collections.defaultdict(dict)
        r = collections.defaultdict(int)
        for idea in ideas:
            start, end = idea[0], idea[1:]
            if re[end].get(start, -1)!=-1:
                re[end][start] += 1
            else:
                re[end][start] = 1
            r[start] += 1
        sums = len(ideas)
        ret = sums ** 2
        print(re, r)
        for key, val in re.items():
            s = sum(val.values())
            keys = list(val.keys())
            for i, k in enumerate(keys):
                a = (r[k]-1)
                for nxt in keys[i+1:]:
                    a += r[nxt]
                ret -= 2 * a
                print(val, a)
        return ret
                

In [58]:
ideas = ["coffee","donuts","time","toffee"]

In [59]:
a = Solution().distinctNames(ideas)
a

defaultdict(<class 'dict'>, {'offee': {'c': 1, 't': 1}, 'onuts': {'d': 1}, 'ime': {'t': 1}}) defaultdict(<class 'int'>, {'c': 1, 'd': 1, 't': 2})
{'c': 1, 't': 1} 2
{'c': 1, 't': 1} 1
{'d': 1} 0
{'t': 1} 1


8

定义 cnt[i][j] 表示组中首字母不包含 i 但包含 j 的组的个数。

In [93]:
class Solution:
    def distinctNames(self, ideas: List[str]) -> int:
        group = defaultdict(int)
        for s in ideas:
            group[s[1:]] |= 1 << (ord(s[0]) - ord('a'))
        ans = 0
        cnt = [[0] * 26 for _ in range(26)]
        for mask in group.values():
            for i in range(26):
                if mask >> i & 1 == 0:
                    for j in range(26):
                        if mask >> j & 1:
                            cnt[i][j] += 1
                else:
                    for j in range(26):
                        if mask >> j & 1 == 0:
                            ans += cnt[i][j]
        return ans * 2